# Clip Raster


In [ ]:
import datetime as dt
import geopandas as gpd
import os
import fiona
import rasterio.mask
from rasterio.fill import fillnodata
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import gdal
import rasterio
import ogr
import warnings
import json
import pandas as pd
from earthpy import clip
from shapely.geometry import JOIN_STYLE

import scipy.spatial
from pathlib import Path
warnings.filterwarnings('ignore')

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

In [ ]:
def clipRaster(raster, polygon):
    kwargs = {'dstNodata':0}
    warped = gdal.Warp('', raster, cutlineDSName=polygon, format ='MEM', **kwargs)
    return warped

# Selecting Datasets
Select the workspace, this is the folder that will be used for the outputs. 
NOTE Select an empty folder as all the files will be deleted from the workspace 

You will also have to select the three datasets used in the analysis. These are:
    
1) Administrative boundaries.

2) Population raster

3) Nighttime lights